how to create a bucket?

create storage bucket?



In [1]:
var qs = require('querystring');
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');

// TODO: set index page

function setPublic(project, bucketName) {
    var client;
    // TODO:
    return authorizeGoogle()
        .then(c => client = c)
        .then(() => client.request({
            method: 'POST',
            url: `https://www.googleapis.com/storage/v1/b/${qs.escape(bucketName)}/acl`,
            data: {
                entity: 'allUsers',
                role: 'READER'
            }
        }))
        .then(() => client.request({
            method: 'POST',
            url: `https://www.googleapis.com/storage/v1/b/${qs.escape(bucketName)}/defaultObjectAcl`,
            data: {
                entity: 'allUsers',
                role: 'READER'
            }
        }))
}

function listBuckets(project, bucketName) {
    var params = {project: project};
    if(bucketName) {
        params.prefix = bucketName;
    }
    return authorizeGoogle()
        .then(client => client.request({
            url: `https://www.googleapis.com/storage/v1/b`,
            params
        }))
        .then(response => response.data.items || [])
}

function addBucket(project, bucketName) {
    return authorizeGoogle()
        .then(client => client.request({
            method: 'POST',
            url: `https://www.googleapis.com/storage/v1/b`,
            params: {project: project},
            data: {
                name: bucketName,
                location: 'us-central1',
                storageClass: 'regional',
                website: {
                    mainPageSuffix: 'index.html'
                }
            }
        }))
        .then(response => response.data.name)
        .then(() => setPublic(project, bucketName))
}

function createBucket(project, bucketName) {
    return listBuckets(project, bucketName)
        .then(buckets => {
            if(buckets.length > 0) {
                console.log(`bucket ${bucketName} already exists`);
                return Promise.resolve(buckets[0].name);
            }
            return addBucket(project, bucketName)
        })
        .then(() => bucketName);
}

module.exports = createBucket;


reading notebook /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[3]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[4]
reading notebook /Users/briancullinan/jupytangular2/Utilities/file system.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Analytics/budgeting.ipynb
reading notebook /Users/briancullinan/jupytangular2/Analytics/calculations for life.ipynb
reading notebook /Users/bria

reading notebook /Users/briancullinan/jupytangular2/Utilities/google calendar.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google contacts.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google maps.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google scheduling.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google search api.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google sheets.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/heartbeat.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/levenshtein.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/mono.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/nmap.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/notebook.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/npm.ipynb
reading notebook /Users/briancullinan/jupytan

[Function: createBucket]

test create bucket?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var createBucket = importer.import('create a bucket');
var project = 'spahaha-ea443';
var bucketName = 'sheet-to-web.sheet-to-web.com';

describe('adding a bucket to google storage', () => {
    it('should add a bucket', () => {
        return createBucket(project, bucketName)
            .then(bucketName => {
                assert(bucketName.length > 0, 'should have added a bucket');
            })
    })
    
});


upload files to google cloud?


In [2]:
var {Storage} = require("@google-cloud/storage");
var mime = require('mime-types');
var path = require('path');
var http = require('http');
var https = require('https');
var importer = require('../Core');
var createBucket = importer.import('create a bucket');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = 'spahaha-ea443';
var storage = new Storage({  
  projectId: project,  
  keyFilename: path.join(PROFILE_PATH, '.credentials/spahaha-ea443-a78ab2269985.json')
});

function streamToGoogle(fileUrl, bucketName, stream, metadata) {
    var gcsname = fileUrl.replace(/\?.*/ig, '');

    return createBucket(project, bucketName)
        .then(() => {
            var bucket = storage.bucket(bucketName);
        
            return new Promise((resolve, reject) => {
                if(typeof stream == 'object') {
                    var files = bucket.file(gcsname);
                    stream.pipe(files.createWriteStream({
                        metadata: Object.assign({
                            contentType: mime.lookup(gcsname) || response.headers['content-type']
                        }, metadata || {})
                    }))
                    .on('error', (err) => {
                        reject(err);
                    })
                    .on('finish', () => {
                        resolve(`https://storage.googleapis.com/${bucketName}/${gcsname}`);
                    });
                } else {
                    var files = bucket.file(path.basename(gcsname));
                    (fileUrl.includes('https://') ? https : http).get(fileUrl, response => {
                        response.pipe(files.createWriteStream({
                            metadata: Object.assign({
                                contentType: mime.lookup(gcsname) || response.headers['content-type']
                            }, metadata || {})
                        }))
                        .on('error', (err) => {
                            reject(err);
                        })
                        .on('finish', () => {
                            resolve(`https://storage.googleapis.com/${bucketName}/${gcsname}`);
                        });
                    });
                }

            });
        });
}

module.exports = streamToGoogle;


compiling /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb[7]


[Function: streamToGoogle]

copy a file in storage bucket?


In [3]:
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');
var project = 'spahaha-ea443';
var qs = require('querystring');

function copyFileBucket(bucket, file) {
    var params = {project: project};
    var client;
    
    return authorizeGoogle()
        .then(c => (client = c))
        .then(() => client.request({
            method: 'POST',
            url: `https://www.googleapis.com/storage/v1/b/${bucket}/o/${qs.escape(file)}/rewriteTo/b/${bucket}/o/${qs.escape(file.replace(/\.html|\.htm/ig, ''))}`,
            params
        }))
}

module.exports = copyFileBucket;


reading notebook /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb
compiling /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb[13]


[Function: copyFileBucket]

test upload files to google storage?


In [ ]:
var path = require('path');
var assert = require('assert');
var importer = require('../Core');
var streamToGoogle = importer.import('upload files google cloud');

describe('upload google storage', () => {
    
    it('should upload a file to a bucket', () => {
        return streamToGoogle(
            'https://www.sgs.com/-/media/global/images/structural-website-images/hero-images/hero-color-palette.jpg?la=en&hash=70B51DB0FA678306B2EAF2E6C4A725BAB0D12342',
            'sheet-to-web.com')
            .then(url => {
                assert(url.length > 0, 'should have a file url');
            })
    })
    
    
})
